In [53]:
import numpy as np
import matplotlib.pyplot as plt
import inspect
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import os

In [54]:
import importlib
import sys
sys.path.append('../')
from Utils.perturbationsnew import RBPIndividual, RBPIndividualNew1, RBPIndividualNew1fast, RBPIndividualNew2, zeroPerturb, noisePerturb, blurPerturb,multisamplebackgroundIdentification
from Utils.models import *
from Utils.data import perturb, generateSynthetic, generateSyntheticPerturbReal,ReadTS

print(sys.path)


['/Users/gali/anaconda3/envs/LIMEtf/lib/python310.zip', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/lib-dynload', '', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages/setuptools/_vendor', '../', '../', '../']


## Test important content removal (accuracy decrease)

### Load synthetic dataset and train CNN model 

In [55]:
x_train, y_train,x_test_original, y_test_original = ReadTS('MedicalImages')
y_train = y_train - 1
y_test_original = y_test_original - 1
print(x_train.shape)
#print(y_train)

(381, 99, 1)


In [56]:
from sklearn.preprocessing import StandardScaler
def normalize(TS):
    original_shape = TS.shape
    
    TS_2d = TS.reshape(original_shape[0] , original_shape[1])
    
    scaler = StandardScaler()
    
    TS_normalized = scaler.fit_transform(TS_2d)
    
    TS = TS_normalized.reshape(original_shape)
    
    return TS

x_train = normalize(x_train)
x_test_original = normalize(x_test_original)
#print(x_train)

In [69]:
model = make_CNN_model(x_train.shape[1:],num_classes = 10)
trained_model, history = train_CNN_model(model, x_train, y_train, epochs=100, batch_size=32)

Epoch 1/100
10/10 [==============================] - 1s 22ms/step - loss: 1.9111 - sparse_categorical_accuracy: 0.4901 - val_loss: 2.1893 - val_sparse_categorical_accuracy: 0.5714
Epoch 2/100
10/10 [==============================] - 0s 12ms/step - loss: 1.6389 - sparse_categorical_accuracy: 0.5296 - val_loss: 2.1235 - val_sparse_categorical_accuracy: 0.5714
Epoch 3/100
10/10 [==============================] - 0s 12ms/step - loss: 1.5066 - sparse_categorical_accuracy: 0.5395 - val_loss: 2.0321 - val_sparse_categorical_accuracy: 0.5714
Epoch 4/100
10/10 [==============================] - 0s 12ms/step - loss: 1.4281 - sparse_categorical_accuracy: 0.5461 - val_loss: 1.9876 - val_sparse_categorical_accuracy: 0.5714
Epoch 5/100
10/10 [==============================] - 0s 12ms/step - loss: 1.3999 - sparse_categorical_accuracy: 0.5428 - val_loss: 1.9067 - val_sparse_categorical_accuracy: 0.5714
Epoch 6/100
10/10 [==============================] - 0s 12ms/step - loss: 1.3669 - sparse_categorica

In [70]:
test_loss, test_acc = trained_model.evaluate(x_test_original, y_test_original)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

24/24 [==============================] - 0s 3ms/step - loss: 1.7228 - sparse_categorical_accuracy: 0.5645
Test accuracy 0.5644736886024475
Test loss 1.7227810621261597


### Create perturbed test sets and calculate decrease in accuracy

In [77]:
def l2norm(perturb):
    difference = perturb - x_test_original
    
    squared_difference = np.square(difference)
    
    sum_of_squares = np.sum(squared_difference, axis=1)
    
    l2_norms = np.sqrt(sum_of_squares)
    
    average_l2_norm = np.mean(l2_norms)
    
    print(average_l2_norm)

    return average_l2_norm


perturb_test_set = x_test_original.copy()
perturb_test_set = perturb_test_set.reshape(x_test_original.shape[0:2])
index_0 = 60
index_1 = 90

def perturb_predict(strategy,index_0,index_1):
    if strategy =='RBP1fast':
        bg = multisamplebackgroundIdentification(perturb_test_set)
        x_test_pert = np.asarray([perturb(strategy,x,index_0,index_1,bg = bg) for x in perturb_test_set])
        x_test_pert = np.expand_dims(x_test_pert, axis=-1)
        predictions_pert = model.predict(x_test_pert)
        predictions_pert = np.argmax(predictions_pert,axis=1)
    else:
        x_test_pert = np.asarray([perturb(strategy,x,index_0,index_1) for x in perturb_test_set])
        x_test_pert = np.expand_dims(x_test_pert, axis=-1)
        predictions_pert = model.predict(x_test_pert)
        predictions_pert = np.argmax(predictions_pert,axis=1)

    accuracy = accuracy_score(y_test_original, predictions_pert)
    precision = precision_score(y_test_original, predictions_pert, average='weighted')  
    recall = recall_score(y_test_original, predictions_pert, average='weighted')
    f1 = f1_score(y_test_original, predictions_pert, average='weighted')
    l2 = l2norm(x_test_pert)

    # print(strategy)
    # print(accuracy)
    # print(precision)
    # print(recall)
    # print(f1)

    return accuracy,precision,recall,f1,l2

In [78]:


strategies = ['RBP', 'RBP1fast', 'zero', 'noise', 'blur']

results = []
l2_results = []

for strategy in strategies:
    accuracy, precision, recall, f1, l2 = perturb_predict(strategy, index_0, index_1)
    results.append([strategy, accuracy, precision, recall, f1])
    l2_results.append(l2)

columns = ['Strategy', 'Accuracy', 'Precision', 'Recall', 'F1 Score']
df_results = pd.DataFrame(results, columns=columns)
l2_columns = ['L2-Norm']
l2_results = pd.DataFrame(l2_results, columns=l2_columns)

predictions_original = model.predict(x_test_original)
predictions_original = np.argmax(predictions_original,axis=1)
accuracy = accuracy_score(y_test_original, predictions_original)
precision = precision_score(y_test_original, predictions_original, average='weighted')  
recall = recall_score(y_test_original, predictions_original, average='weighted')
f1 = f1_score(y_test_original, predictions_original, average='weighted')
original_data = ['original', accuracy, precision, recall, f1]
df_results.loc[-1] = original_data  
df_results.index = df_results.index + 1  
df_results = df_results.sort_index()  

df_results.loc[2, 'Strategy'] = 'MRBP'

current_directory = os.getcwd()
result_folder = os.path.join(current_directory, 'result_data')

csv_path = os.path.join(result_folder, 'MedicalImages.csv')
df_results.to_csv(csv_path, index=False)

csv_path = os.path.join(result_folder, 'MedicalImagesl2.csv')
l2_results.to_csv(csv_path, index=False)

print("done")
print(df_results)
print(l2_results)

24/24 [==============================] - 0s 3ms/step
3.327621980007752
24/24 [==============================] - 0s 3ms/step
4.463439265715238
 1/24 [>.............................] - ETA: 0s

/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


24/24 [==============================] - 0s 3ms/step
4.005427218077773
24/24 [==============================] - 0s 3ms/step
8.222989392756915
21/24 [=========================>....] - ETA: 0s

/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


24/24 [==============================] - 0s 3ms/step
0.23326514726403366
24/24 [==============================] - 0s 3ms/step
done
   Strategy  Accuracy  Precision    Recall  F1 Score
0  original  0.564474   0.457396  0.564474  0.437506
1       RBP  0.538158   0.407771  0.538158  0.393090
2      MRBP  0.467105   0.339253  0.467105  0.384027
3      zero  0.534211   0.403018  0.534211  0.386154
4     noise  0.393421   0.423203  0.393421  0.372780
5      blur  0.552632   0.391789  0.552632  0.416148
    L2-Norm
0  3.327622
1  4.463439
2  4.005427
3  8.222989
4  0.233265


/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
